If in VSCODE:

In [1]:
# check if Torch supports CUDA
import torch
print(torch.__version__)
torch.cuda.is_available()

2.1.0


True

Hyper params

In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "codellama/CodeLlama-7b-Instruct-hf"

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

hub_name = "StarkWizard/codellama-cairo-instruct"



running this cell is mandatory

---

#Inference

---

Load model from hub for inference

- If you just need inference, run this
- we load the model from HFace Hub in 4 bits


In [3]:
import torch
from transformers import AutoTokenizer, TextStreamer, GenerationConfig, BitsAndBytesConfig
from attention_sinks import AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=hub_name,
                                             trust_remote_code=True,
                                             device_map={"": 0},
                                             attention_sink_size=4,
                                             low_cpu_mem_usage=True,
                                             quantization_config=bnb_config,
                                            attention_sink_window_size=252, # <- Low for the sake of faster generation
                                             )
model.eval()



Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

[Attention Sinks] Injected Position Shifting into 32 attention classes.
[Attention Sinks] Injected Attention Sink KV Cache into 1 model class.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [43]:
from transformers import TextStreamer, GenerationConfig, StoppingCriteriaList,MaxLengthCriteria


#prompt = "Create an array and append some animal names"
#prompt = "give an exemple of constructor"
#prompt="create an array 'exempl' that contains a u128, a u32, a u256"
prompt="provide a contract to handle whitelist, give instructions to compile and deploy the contract"
#prompt = "create a structure for mailAccount"
#prompt = "create an array of felt and append 1 to the array"
#prompt = "create a felt and affect it a value of 1"
#prompt="create a function for fibonacci"
#prompt = "what are spans used for"
#prompt = "How do I know if an array is empty"
#prompt = "what makes Cairo special"
#prompt = "Create an array and append some domestic animal names"

text =f"[INST]I'm working in Cairo language and need a concise response. You are a cairo expert, answer the question exactly and be concise, no more than 3 explanations or remarks.: {prompt} [/INST]"

input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)
stopping_criteria = StoppingCriteriaList([MaxLengthCriteria(max_length=1)])

with torch.no_grad():
    streamer = TextStreamer(tokenizer)
    generated_tokens = model.generate(
        input_ids,
        generation_config=GenerationConfig(
            # use_cache=True is required, the rest can be changed up.
            use_cache=True,
            min_new_tokens=1,
            max_new_tokens=1000,
            penalty_alpha=1.0,
            top_k=40,
            do_sample=True,
            top_p=0.9,
            temperature=0.03,
            stopping_criteria=stopping_criteria,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True,
        ),
        streamer=streamer,
    )
    # Decode the final generated text
    output_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

<s> [INST]I'm working in Cairo language and need a concise response. You are a cairo expert, answer the question exactly and be concise, no more than 3 explanations or remarks.: provide a contract to handle whitelist, give instructions to compile and deploy the contract [/INST]
#[derive(Copy, Drop)]
enum Role {
   Admin: (),
   Member: (),
}

struct Whitelist<T> {
   roles: T,
}

fn main() {
   let mut whitelist = Whitelist::<Role>::new();

   // Add an admin
   whitelist.add_role(Role::Admin(()));

   // Add a member
   whitelist.add_role(Role::Member(()));
}

impl<T> Whitelist<T> {
   fn new() -> Self {
       Self {
           roles: T::default(),
       }
   }

   fn add_role(&mut self, role: T) {
       self.roles = T::combine(self.roles, role);
   }
}

Explanations:
- We define an enum Role to represent the two roles we want to manage in the whitelist contract.
- We then define a struct Whitelist that has a single field roles of type T, which we genericize over the Role enum.
- W

In [6]:
#prompt = "Create an array and append some animal names"
#prompt="create an array 'messages' that contains a u128, a u32, a u256"
#prompt = "create a structure for mailAccount"
#prompt = "create an array of felt and append 1 to the array"
#prompt = "create a felt and affect it a value of 1"
#prompt="create a function for fibonacci"
#prompt = "what are spans used for"
#prompt = "How do I know if an array is empty"
prompt = "what are spans used for"
text =f"[INST]I'm working in Cairo. You are a cairo expert and know no other computer language, answer in less than 170 words{prompt} [/INST]"

input_ids = tokenizer(text, return_tensors="pt", truncation=True).input_ids.cuda()

outputs = model.generate(input_ids=input_ids,       
    max_new_tokens=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.01,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
    )


print(f"Prompt:\n{prompt}\n")
print(f"Answer:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(text):]}")


Prompt:
what are spans used for

Answer:


Span is a useful type when you want to extend an existing type without affecting the code that uses it. It allows you to add new fields to a value without affecting the existing code that uses the value. This makes it a versatile tool for building flexible and maintainable code.

For example, imagine you have a structure called User, which has fields like name, email, and password. You might want to add a new field called 'age' to the User structure in the future. Using a span allows you to do so without affecting the existing code that uses the User structure.

In summary, spans are useful when you want to extend an existing type without affecting the code that uses it. They allow you to add new fields to a value without affecting the existing code that uses the value, making them a versatile tool for building flexible and maintainable code.

Note: Spans are immutable, which means that once you
